<a href="https://colab.research.google.com/github/m-newhauser/recipes/blob/main/rag_over_pdfs_docling_weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing RAG over PDFs with Weaviate and Docling
## A recipe 🧑‍🍳 🐥 💚
By Mary Newhauser, MLE @ Weaviate

This is a code recipe that uses [Weaviate](https://weaviate.io/) to perform RAG over PDF documents parsed by [Docling](https://ds4sd.github.io/docling/).

In this notebook, we accomplish the following:
* Parse the top machine learning papers on [arXiv](https://arxiv.org/) using Docling
* Perform hierarchical chunking of the documents using Docling
* Generate text embeddings with OpenAI
* Perform RAG using [Weaviate](https://weaviate.io/developers/weaviate/search/generative)

To run this notebook, you'll need:
* An [OpenAI API key](https://platform.openai.com/docs/quickstart)
* Access to GPU/s

Note: For best results, please run this notebook in Google Colab using a T4 GPU.

### Install Docling and Weaviate client

Note: If Colab prompts you to restart the session after running the cell below, click "restart" and proceed with running the rest of the notebook.

In [1]:
%%capture
%pip install docling=="2.7.0"
%pip install -U weaviate-client=="4.9.4"

## 🐥 Part 1: Docling

In [2]:
import tensorflow as tf

# Check that GPU is enabled
assert 'GPU' in [device.device_type for device in tf.config.list_physical_devices()], "GPU is not enabled. Please go to Runtime > Change runtime type > Hardware accelerator and select GPU."

Here, we've collected 10 influential machine learning papers published as PDFs on arXiv. Because Docling does not yet have title extraction for PDFs, we manually add the titles in a corresponding list.

Note: Converting all 10 papers should take around 8 minutes with a T4 GPU.

In [3]:
# Influential machine learning papers
source_urls = [
    "https://arxiv.org/pdf/1706.03762",
    "https://arxiv.org/pdf/1810.04805",
    "https://arxiv.org/pdf/1406.2661",
    "https://arxiv.org/pdf/1409.0473",
    "https://arxiv.org/pdf/1412.6980",
    "https://arxiv.org/pdf/1312.6114",
    "https://arxiv.org/pdf/1312.5602",
    "https://arxiv.org/pdf/1512.03385",
    "https://arxiv.org/pdf/1409.3215",
    "https://arxiv.org/pdf/1301.3781"
]

# And their corresponding titles (because Docling doesn't have title extraction yet!)
source_titles = [
    "Attention Is All You Need",
    "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding",
    "Generative Adversarial Nets",
    "Neural Machine Translation by Jointly Learning to Align and Translate",
    "Adam: A Method for Stochastic Optimization",
    "Auto-Encoding Variational Bayes",
    "Playing Atari with Deep Reinforcement Learning",
    "Deep Residual Learning for Image Recognition",
    "Sequence to Sequence Learning with Neural Networks",
    "A Neural Probabilistic Language Model"
]

### Convert PDFs to Docling documents

Here we use Docling's `.convert_all()` to parse a batch of PDFs. The result is a list of Docling documents that we can use for text extraction.

In [4]:
from docling.datamodel.document import ConversionResult
from docling.document_converter import DocumentConverter

# Instantiate the doc converter
doc_converter = DocumentConverter()

# Directly pass list of files or streams to `convert_all`
conv_results_iter = doc_converter.convert_all(source_urls) # previously `convert`

# Iterate over the generator to get a list of Docling documents
docs = [result.document for result in conv_results_iter]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

otslp_all_standard_094_clean.check:   0%|          | 0.00/213M [00:00<?, ?B/s]

otslp_all_fast.check:   0%|          | 0.00/146M [00:00<?, ?B/s]

model.pt:   0%|          | 0.00/202M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

...

.gitignore:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

(…)artifacts/tableformer/fat/tm_config.json:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

(…)del_artifacts/tableformer/tm_config.json:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


### Post-process extracted document data
#### Perform hierarchical chunking on documents

We use Docling's `HierarchicalChunker()` to perform hierarchy-aware chunking of our list of documents. This is meant to preserve some of the structure and relationships within the document, which enables more accurate and relevant retrieval in our RAG pipeline.

In [5]:
from docling_core.transforms.chunker import HierarchicalChunker

# Initialize lists for text, and titles
texts, titles = [], []

chunker = HierarchicalChunker()

# Process each document in the list
for doc, title in zip(docs, source_titles):       # Pair each document with its title
    chunks = list(chunker.chunk(doc))             # Perform hierarchical chunking and get text from chunks
    for chunk in chunks:
        texts.append(chunk.text)
        titles.append(title)

Because we're splitting the documents into chunks, we'll concatenate the article title to the beginning of each chunk for additional context.

In [6]:
# Concatenate title and text
for i in range(len(texts)):
    texts[i] = f"{titles[i]} {texts[i]}"

## 💚 Part 2: Weaviate
### Create and configure an embedded Weaviate collection

We'll be using the OpenAI API for both generating the text embeddings and for the generative model in our RAG pipeline. The code below dynamically fetches your API key based on whether you're running this notebook in Google Colab and running it as a regular Jupyter notebook. All you need to do is replace `openai_api_key_var` with the name of your environmental variable name or Colab secret name for the API key.

If you're running this notebook in Google Colab, make sure you [add](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) your API key as a secret.

In [7]:
# OpenAI API key variable name
openai_api_key_var = "OPENAI_APIKEY"  # Replace with the name of your secret/env var

# Fetch OpenAI API key
try:
    # If running in Colab, fetch API key from Secrets
    import google.colab
    from google.colab import userdata
    openai_api_key = userdata.get(openai_api_key_var)
    if not openai_api_key:
        raise ValueError(f"Secret '{openai_api_key_var}' not found in Colab secrets.")
except ImportError:
    # If not running in Colab, fetch API key from environment variable
    import os
    openai_api_key = os.getenv(openai_api_key_var)
    if not openai_api_key:
        raise EnvironmentError(
            f"Environment variable '{openai_api_key_var}' is not set. "
            "Please define it before running this script."
        )

[Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded) allows you to spin up a Weaviate instance directly from your application code, without having to use a Docker container. If you're interested in other deployment methods, like using Docker-Compose or Kubernetes, check out this [page](https://weaviate.io/developers/weaviate/installation) in the Weaviate docs.

In [8]:
import weaviate

# Connect to Weaviate embedded
client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": openai_api_key
    }
)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6986


In [9]:
import weaviate.classes.config as wc
from weaviate.classes.config import Property, DataType

# Define the collection name
collection_name = "docling"

# Delete the collection if it already exists
if (client.collections.exists(collection_name)):
    client.collections.delete(collection_name)

# Create the collection
collection = client.collections.create(
    name=collection_name,
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-large",                           # Specify your embedding model here
    ),

    # Enable generative model from Cohere
    generative_config=wc.Configure.Generative.openai(
    model="gpt-4o"                                                # Specify your generative model for RAG here
    ),

    # Define properties of metadata
    properties=[
        wc.Property(
            name="text",
            data_type=wc.DataType.TEXT
        ),
        wc.Property(
            name="title",
            data_type=wc.DataType.TEXT,
            skip_vectorization=True
        ),
    ]
)

### Wrangle data into an acceptable format for Weaviate

Transform our data from lists to a list of dictionaries for insertion into our Weaviate collection.

In [10]:
# Initialize the data object
data = []

# Create a dictionary for each row by iterating through the corresponding lists
for text, title in zip(texts, titles):
    data_point = {
        "text": text,
        "title": title,
    }
    data.append(data_point)

### Insert data into Weaviate and generate embeddings

Embeddings will be generated upon insertion to our Weaviate collection.

In [11]:
# Insert text chunks and metadata into vector DB collection
response = collection.data.insert_many(
    data
)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


### Query the data

Here, we perform a simple similarity search to return the most similar embedded chunks to our search query.

In [12]:
from weaviate.classes.query import MetadataQuery

response = collection.query.near_text(
    query="bert",
    limit=2,
    return_metadata=MetadataQuery(distance=True),
    return_properties=["text", "title"]
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'text': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding A distinctive feature of BERT is its unified architecture across different tasks. There is mini-', 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'}
0.6580133438110352
{'text': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding We introduce a new language representation model called BERT , which stands for B idirectional E ncoder R epresentations from T ransformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substa

### Perform RAG on parsed articles

Weaviate's `generate` module allows you to perform RAG over your embedded data without having to use a separate framework.

We specify a prompt that includes the field we want to search through in the database (in this case it's `text`), a query that includes our search term, and the number of retrieved results to use in the generation.

In [13]:
from rich.console import Console
from rich.panel import Panel

# Create a prompt where context from the Weaviate collection will be injected
prompt = "Explain how {text} works, using only the retrieved context."
query = "bert"

response = collection.generate.near_text(
    query=query,
    limit=3,
    grouped_task=prompt,
    return_properties=["text", "title"]
)

# Prettify the output using Rich
console = Console()

console.print(Panel(f"{prompt}".replace("{text}", query), title="Prompt", border_style="bold red"))
console.print(Panel(response.generated, title="Generated Content", border_style="bold green"))

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│ Explain how bert works, using only the retrieved context.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Generated Content ───────────────────────────────────────────────╮
│ BERT, which stands for Bidirectional Encoder Representations from Transformers, is a language representation    │
│ model designed to pretrain deep bidirectional representations from unlabeled text. It does this by jointly      │
│ conditioning on both left and right context in all layers, unlike traditional left-to-right or right-to-left    │
│ language models. This pre-training approach allows BERT to be fine-tuned with just one additional output layer  │
│ to create state-of-the-art models for various tasks, such as question answering and language inference, without │
│ needing substantial task-specific architecture modifications. BERT's unified architecture is applicable across  │
│ different tasks, making it a versatile tool in language understanding.                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
# Create a prompt where context from the Weaviate collection will be injected
prompt = "Explain how {text} works, using only the retrieved context."
query = "a generative adversarial net"

response = collection.generate.near_text(
    query=query,
    limit=3,
    grouped_task=prompt,
    return_properties=["text", "title"]
)

# Prettify the output using Rich
console = Console()

console.print(Panel(f"{prompt}".replace("{text}", query), title="Prompt", border_style="bold red"))
console.print(Panel(response.generated, title="Generated Content", border_style="bold green"))

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│ Explain how a generative adversarial net works, using only the retrieved context.                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Generated Content ───────────────────────────────────────────────╮
│ The provided context does not contain specific information on how BERT works. However, based on general         │
│ knowledge, BERT (Bidirectional Encoder Representations from Transformers) is a model designed to understand the │
│ context of words in a sentence by looking at both the left and right sides of a word simultaneously. It is      │
│ pre-trained on a large corpus of text data using two unsupervised tasks: Masked Language Modeling (MLM) and     │
│ Next Sentence Prediction (NSP). In MLM, some words in a sentence are masked, and the model learns to predict    │
│ them based on the surrounding context. In NSP, the model learns to predict if a given sentence is likely to     │
│ follow another sentence. This pre-training allows BERT to capture deep bidirectional representations, making it │
│ effective for various natural language processing tasks.                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

We can see that our RAG pipeline performs relatively well for simple queries, especially given the small size of the dataset. Scaling this method for converting a larger sample of PDFs would require more compute (GPUs) and a more advanced deployment of Weaviate (like Docker, Kubernetes, or Weaviate Cloud). However, this notebook demonstrates that Docling is a robust and powerful open source tool for converting PDFs to structured data.

To take this solution to next level, you consider:
* Experimenting with different [chunking techniques](https://weaviate.io/developers/academy/py/standalone/chunking/introduction)
* Using a RAG framework like [DSPy](https://weaviate.io/developers/integrations/llm-frameworks/dspy), [LlamaIndex](https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/weaviate/), or [LangChain](https://python.langchain.com/docs/integrations/vectorstores/weaviate/)
* Implementing [advanced RAG](https://weaviate.io/blog/advanced-rag) techniques, like [Agentic RAG](https://weaviate.io/blog/what-is-agentic-rag)